In [1]:
import requests
import time


def getMassData(lastDay=1):
    url = 'https://api.binance.com/api/v3/klines'
    current_time = int(time.time() * 1000)
    startDay = current_time - ((lastDay+1000) * 24 * 60 * 60 * 1000)
    endDay = current_time - (lastDay * 24 * 60 * 60 * 1000)

    params = {
    'symbol': 'BTCUSDT',      # The trading pair you're interested in
    'interval': '1d',         # 1-hour candlestick data (hourly prices)
    'startTime': startDay,  # Data from 30 days ago
    'endTime': endDay,    # Data until 20 days ago
    }

    response = requests.get(url, params=params)
    data = response.json()

    result=[]

    for candle in data:
        close_price = candle[4]
        result.append(close_price)

    return result

In [5]:
import csv

import pandas as pd

data = getMassData()

test_data = data[-10:]
  
train_data = data[:-10]


df = pd.DataFrame(data)

df.to_csv('data/massData.csv', index=False, header=False)

In [20]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

# Example data (replace with your actual data)
data = train_data  # Replace with your data

# Ensure the data is fully numeric and clean up any non-numeric entries
# Convert data to a Pandas Series first
df = pd.Series(data)

# Attempt to convert all values to numeric, coercing non-numeric values to NaN
df = pd.to_numeric(df, errors='coerce')

# Drop any NaN values that were caused by non-numeric entries
df = df.dropna()

# Check if the conversion and cleaning worked
if df.isnull().values.any():
    raise ValueError("The data still contains non-numeric values after conversion.")

# Fit the SARIMAX model (adjust p, d, q, and seasonal parameters as needed)
model = SARIMAX(df, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))  # Adjust the order if necessary
model_fit = model.fit(disp=False)

# Forecast the next 10 values
forecast = model_fit.forecast(steps=10)

# Print the forecasted values
print(forecast.to_list())
print(test_data)




[28539.707808494368, 28149.808164341925, 27767.881572435646, 28350.584766295648, 28245.86766097462, 28229.689038804743, 28596.146069652885, 28629.223381387277, 29081.883632197452, 29175.360884236645]
['28068.26000000', '28669.86000000', '29026.16000000', '28838.16000000', '29505.61000000', '28848.20000000', '28430.10000000', '27668.79000000', '27628.27000000', '27598.75000000']


In [26]:
forecast_list = forecast.to_list()


list1 = [float(value) for value in forecast_list]
list2 = [float(value) for value in test_data]


for i in range(len(list1)):
    print(int(list1[i]) - int(list2[i]))

471
-520
-1259
-488
-1260
-619
166
961
1453
1577
